# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.preprocessing import Binarizer
clf = Binarizer(threshold=2)
clf.fit(df[metadata['features']].values)


Binarizer(threshold=2)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lOutputs;
	}

	tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {

		tTable lTable;

		lTable["Feature_0"] = { ( Feature_0 <= 2 ?  0 : 1 ) };
		lTable["Feature_1"] = { ( Feature_1 <= 2 ?  0 : 1 ) };
		lTable["Feature_2"] = { ( Feature_2 <= 2 ?  0 : 1 ) };
		lTable["Feature_3"] = { ( Feature_3 <= 2 ?  0 : 1 ) };

		return lTable;
	}

	tTable compute_model_outputs_from_table( tTable const & iTable) {
		tTable lTable = compute_features(iTable.at("Feature_0")[0], iTable.at("Feature_1")[0], iTable.at("Feature_2")[0], iTable.at("Feature_3")[0]);

		return lTable;
	}

} // eof namespace 



In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_139920226301552.exe /tmp/sklearn2sql_cpp_139920226301552.cpp'
idx,Feature_0,Feature_1,Feature_2,Feature_3
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0
5,1,1,0,0
6,1,1,0,0
7,1,1,0,0
8,1,1,0,0
9,1,1,0,0
10,1,1,0,0
11,1,1,0,0
12,1,1,0,0
13,1,1,0,0
14,1,1,0,0
15,1,1,0,0
16,1,1,0,0
17,1,1,0,0
18,1,1,0,0
19,1,1,0,0
20,1,1,0,0
21,1,1,0,0
22,1,1,0,0
23,1,1,0,0
24,1,1,0,0
25,1,1,0,0
26,1,1,0,0
27,1,1,0,0
28,1,1,0,0
29,1,1,0,0
30,1,1,0,0
31,1,1,0,0
32,1,1,0,0
33,1,1,0,0
34,1,1,0,0
35,1,1,0,0
36,1,1,0,0
37,1,1,0,0
38,1,1,0,0
39,1,1,0,0
40,1,1,0,0
41,1,1,0,0
42,1,1,0,0
43,1,1,0,0
44,1,1,0,0
45,1,1,0,0
46,1,1,0,0
47,1,1,0,0
48,1,1,0,0
49,1,1,0,0
50,1,1,1,0
51,1,1,1,0
52,1,1,1,0
53,1,1,1,0
54,1,1,1,0
55,1,1,1,0
56,1,1,1,0
57,1,1,1,0
58,1,1,1,0
59,1,1,1,0
60,1,0,1,0
61,1,1,1,0
62,1,1,1,0
63,1,1,1,0
64,1,1,1,0
65,1,1,1,0
66,1,1,1,0
67,1,1,1,0
68,1,1,1,0
69,1,1,1,0
70,1,1,1,0
71,1,1,1,0
72,1,1,1,0
73,1,1,1,0
74,1,1,1,0

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,1,1,1,1
74,74,1,1,1,0
9,9,1,1,0,0
88,88,1,1,1,0
25,25,1,1,0,0
5,5,1,1,0,0
48,48,1,1,0,0
117,117,1,1,1,1
83,83,1,1,1,0
105,105,1,1,1,1


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,1.0,1.0,1.0,1.0
74,74,1.0,1.0,1.0,0.0
9,9,1.0,1.0,0.0,0.0
88,88,1.0,1.0,1.0,0.0
25,25,1.0,1.0,0.0,0.0
5,5,1.0,1.0,0.0,0.0
48,48,1.0,1.0,0.0,0.0
117,117,1.0,1.0,1.0,1.0
83,83,1.0,1.0,1.0,0.0
105,105,1.0,1.0,1.0,1.0


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Feature_0_skl,Feature_1_skl,Feature_2_skl,Feature_3_skl,idx_cpp,Feature_0_cpp,Feature_1_cpp,Feature_2_cpp,Feature_3_cpp
114,114,1.0,1.0,1.0,1.0,114,1,1,1,1
74,74,1.0,1.0,1.0,0.0,74,1,1,1,0
9,9,1.0,1.0,0.0,0.0,9,1,1,0,0
88,88,1.0,1.0,1.0,0.0,88,1,1,1,0
25,25,1.0,1.0,0.0,0.0,25,1,1,0,0
5,5,1.0,1.0,0.0,0.0,5,1,1,0,0
48,48,1.0,1.0,0.0,0.0,48,1,1,0,0
117,117,1.0,1.0,1.0,1.0,117,1,1,1,1
83,83,1.0,1.0,1.0,0.0,83,1,1,1,0
105,105,1.0,1.0,1.0,1.0,105,1,1,1,1


In [14]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())

count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
